In [1]:
import platform
import sys
import pandas as pd
import calendar
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import functions.preprocessing as prep

import sklearn
from sklearn import svm
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import sklearn.metrics as sm
from sklearn import datasets
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print("This jupyter notebook is running on Python " + platform.python_version())
# It's good practice to assert packages requirements at the beginning of a script:
assert sys.version_info >= (3, 6)  # Tested with Python==3.7.9

This jupyter notebook is running on Python 3.7.9


In [56]:
training_set_file_path = 'data/training_set_VU_DM.csv'
test_set_file_path = 'data/test_set_VU_DM.csv'
training_data = pd.read_csv(training_set_file_path)
test_data = pd.read_csv(test_set_file_path)

In [57]:
training_data = prep.parseDateTime(training_data)
training_data, useless_columns = prep.removeUselessColumns(training_data, threshold=90)
training_data = prep.replaceNaN(training_data)
training_data = prep.mergeComps(training_data)
#training_data.info(verbose=True)

c:\Users\Ramon\Desktop\DMT_Asg_2\functions\preprocessing.py:19: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  dataFrame = dataFrame.drop(useless_columns, 1)
c:\Users\Ramon\Desktop\DMT_Asg_2\functions\preprocessing.py:20: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  dataFrame = dataFrame.drop(['comp3_rate','comp3_inv'], 1)


Here we filter some of our data. Removing useless columns etc.

In [58]:
columns_to_drop = ['price_usd', 'month', 'year', 'srch_adults_count', 'srch_children_count', 'prop_id', 
                   'srch_room_count', 'prop_location_score2', 'prop_review_score', 'orig_destination_distance',]
print(useless_columns)
temp = useless_columns[:-1]
print(temp)
#training_data = training_data.drop(columns_to_drop, 1)
#training_data.info(verbose=True)


['visitor_hist_starrating', 'visitor_hist_adr_usd', 'srch_query_affinity_score', 'comp1_rate', 'comp1_inv', 'comp1_rate_percent_diff', 'comp3_rate_percent_diff', 'comp4_rate', 'comp4_inv', 'comp4_rate_percent_diff', 'comp6_rate', 'comp6_inv', 'comp6_rate_percent_diff', 'comp7_rate', 'comp7_inv', 'comp7_rate_percent_diff', 'gross_bookings_usd']
['visitor_hist_starrating', 'visitor_hist_adr_usd', 'srch_query_affinity_score', 'comp1_rate', 'comp1_inv', 'comp1_rate_percent_diff', 'comp3_rate_percent_diff', 'comp4_rate', 'comp4_inv', 'comp4_rate_percent_diff', 'comp6_rate', 'comp6_inv', 'comp6_rate_percent_diff', 'comp7_rate', 'comp7_inv', 'comp7_rate_percent_diff']


In [59]:
#training_data_y = training_data.booking_bool
#training_data_x = training_data.drop(['booking_bool'],1)
training_data_y = training_data.booking_bool
training_data_x = training_data.drop(['click_bool', 'position', 'booking_bool'],1)
training_data_x['srch_id'] = training_data_x['srch_id'].astype('category')
search_id = training_data_x.srch_id.tolist()
training_data_x = training_data_x.drop(['srch_id'],1)
training_data_x = training_data_x.drop(columns_to_drop,1)

training_data_x.info(verbose=True)
scaler = StandardScaler().fit(training_data_x)
training_data_x = scaler.transform(training_data_x)

search_id_test = test_data.srch_id.tolist()
prop_id_test = test_data.prop_id.tolist()
test_data = prep.parseDateTime(test_data)
test_data = test_data.drop(useless_columns[:-1],1)
test_data = prep.replaceNaN(test_data)
test_data = prep.mergeComps(test_data)
test_data = test_data.drop(columns_to_drop,1)
test_data = test_data.drop(['comp3_rate','comp3_inv'], 1)
test_data = test_data.drop(['srch_id'],1)
test_data.info(verbose=True)

#training_data_x, testing_data_x, training_data_y, testing_data_y = train_test_split(training_data_x, training_data_y, test_size=0.25, random_state=42)

del training_data
#columns_to_drop = ['price_usd', 'month', 'year', 'srch_adults_count', 'srch_children_count', 'prop_id', 'srch_room_count']


c:\Users\Ramon\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  after removing the cwd from sys.path.
c:\Users\Ramon\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  import sys
c:\Users\Ramon\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4958347 entries, 0 to 4958346
Data columns (total 16 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   site_id                      int64  
 1   visitor_location_country_id  int64  
 2   prop_country_id              int64  
 3   prop_starrating              int64  
 4   prop_brand_bool              int64  
 5   prop_location_score1         float64
 6   prop_log_historical_price    float64
 7   promotion_flag               int64  
 8   srch_destination_id          int64  
 9   srch_length_of_stay          int64  
 10  srch_booking_window          int64  
 11  srch_saturday_night_bool     int64  
 12  random_bool                  int64  
 13  comp_rate                    float64
 14  comp_inv                     float64
 15  comp_rate_percent_diff       float64
dtypes: float64(5), int64(11)
memory usage: 605.3 MB


c:\Users\Ramon\Desktop\DMT_Asg_2\functions\preprocessing.py:9: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  return dataFrame.drop('date_time', 1)
c:\Users\Ramon\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
c:\Users\Ramon\Desktop\DMT_Asg_2\functions\preprocessing.py:33: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  'comp2_rate_percent_diff', 'comp5_rate_percent_diff', 'comp8_rate_percent_diff'], 1)
c:\Users\Ramon\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:20: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
c:\Users\Ramon\AppData\Loc

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4959183 entries, 0 to 4959182
Data columns (total 16 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   site_id                      int64  
 1   visitor_location_country_id  int64  
 2   prop_country_id              int64  
 3   prop_starrating              int64  
 4   prop_brand_bool              int64  
 5   prop_location_score1         float64
 6   prop_log_historical_price    float64
 7   promotion_flag               int64  
 8   srch_destination_id          int64  
 9   srch_length_of_stay          int64  
 10  srch_booking_window          int64  
 11  srch_saturday_night_bool     int64  
 12  random_bool                  int64  
 13  comp_rate                    float64
 14  comp_inv                     float64
 15  comp_rate_percent_diff       float64
dtypes: float64(5), int64(11)
memory usage: 605.4 MB


c:\Users\Ramon\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:22: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only


We then create and train the model

In [67]:
clf = svm.LinearSVC() 
clf.fit(training_data_x, training_data_y)

c:\Users\Ramon\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\svm\_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC()

Make the predictions, and put them into a dataframe

In [68]:
predicted_y = clf.predict(test_data)
predicted_y_probability = clf._predict_proba_lr(test_data)

In [76]:
print(predicted_y[1990:2000], predicted_y_probability[1990:2000])

[1 1 1 1 1 0 0 0 0 0] [[9.70535011e-04 9.99029465e-01]
 [9.72756229e-04 9.99027244e-01]
 [9.69884234e-04 9.99030116e-01]
 [9.71849478e-04 9.99028151e-01]
 [9.48583494e-04 9.99051417e-01]
 [6.29215761e-01 3.70784239e-01]
 [6.35069805e-01 3.64930195e-01]
 [6.34567238e-01 3.65432762e-01]
 [6.34649757e-01 3.65350243e-01]
 [6.35966912e-01 3.64033088e-01]]


In [77]:
highest_predict = np.amax(predicted_y_probability, axis=1)
myDataFrame = pd.DataFrame(list(zip(search_id_test, prop_id_test, predicted_y, highest_predict)), columns=['srch_id', 'prop_id', 'predicted_y', 'predicted_y_probability'])

In [79]:
myDataFrame = myDataFrame.sort_values(['srch_id', 'predicted_y_probability'], ascending = [True, False])
myDataFrame = myDataFrame[myDataFrame.predicted_y == 1]
myDataFrame[0:5]

,srch_id,prop_id,predicted_y,predicted_y_probability
25,1,128085,1,0.992293
7,1,37567,1,0.992230
23,1,99484,1,0.992189
9,1,54937,1,0.992188
12,1,61934,1,0.992161


In [81]:
outputFrame = myDataFrame.drop(['predicted_y', 'predicted_y_probability'],1)
outputFrame.to_csv('testOutput2.csv', index=False)

c:\Users\Ramon\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.
